In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
class BBoxSimilaritySearch(nn.Module):
    def __init__(self, learner):
        super().__init__()
        self.learner = learner
        
    def forward(self, exemplar, search_target, n=5):
#         exe_embed = self.embedder(exemplar)
#         target_embed = self.embedder(search_target)
#         ans = self.searcher(exe_embed, target_embed)
        # hmm, maybe one backbone and multiple heads?
        # would Centernet itself work well here?
        # concat CenterNet features with Siamese?
        # recurrent bbox controller?
        #  adaptive computation time?
        # higher order training?
        w, h = exemplar.size(2), exemplar.size(1)
        windows = [(x, y, x+w, y+h) for x, y in zip(range(0, search_target.size(1), w), range(0, search_target.size(2), h))]
        crops = list(map(lambda bbox: search_target.crop(bbox), windows))
        candidates = find_nneighbours(self.learner, exemplar, crops, n=n)
        
        return candidates